In [ ]:
GLOBAL_LOADED_ATOMRNN=True

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%run /home/venkata/python/python_libraries/CADD/PDBReader.ipynb

import sys,os
import time

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence,pad_sequence

DEVICE=torch.device("cuda")

Loaded Library 'General (RDKit)'
PDB Tools Loaded
cuda


In [4]:
# Loading the PDB file as string
def readPDBAtomSeq(filename,skipHs=False):
    with open(filename, 'r') as f:
        lines = f.readlines()
    atypes=[ln[12:16].strip() for ln in lines if (ln[0:4]=="ATOM" or ln[0:6]=="HETATM") and ((skipHs and ln[12:16].strip()[0]!="H") or (not skipHs))]
    return atypes

def readPDBFiles(files,N,S,skipHs=False):
    rawdata=[]
    K=0
    L=0
    print("Target:",N)
    #Skip first 'S', and read 'N' files
    file_names = [fn for fn in files]
    sorted(file_names)
    for f in file_names:
        if K<S:
            K+=1
            continue
        if (L+1)%2500==0: print(L+1,f)
        rawdata.append([OPENFLAG]+readPDBAtomSeq(folder+"/"+f,skipHs=skipHs))
        L+=1
        if L>=N: break
    print(len(rawdata),"files read")
    return rawdata
def samplechar(poss):
  posses=np.sum(poss)
  if posses<1e-10: return np.random.choice(range(len(poss)))
  else:
    poss=np.nan_to_num(poss,0.)
    poss/=sum(poss)
  return np.random.choice(range(len(poss)),p=poss)
def buildVocabulary(asets):
    global MASK
    vocab=set()
    for aset in asets:
        vocab=set.union(vocab,set(aset))
    return vocab

In [3]:
# Some default parameters
HIDE=""
MASK='\0'
SPLIT="~"
OPENFLAG="^"
MAXLEN=72
BUFFER_SIZE=1000
DTYPE_INT=torch.int64

datafunc=readPDBAtomSeq

In [5]:
def encodeAll(seqlist,keys=dict(),intlevel=torch.int32,strict=False):
    maxint=max([keys[k] for k in keys.keys()],default=0)
    nextint=maxint+1
    ret=[]
    for seq in seqlist:
        intseq=[]
        for el in seq:
            if el in keys.keys(): intseq.append(keys[el])
            else:
                if strict: raise ValueError("Key not found: "+str(el))
                intseq.append(nextint)
                keys[el]=nextint
                nextint+=1
        ret.append(torch.tensor(intseq,dtype=intlevel).to(DEVICE))
    return ret,keys
def rawdataToDataset(rawdata,keys=dict(),intlevel=DTYPE_INT,shuffle=True):
    rawdata_encoded,encoded_keys=encodeAll(rawdata,keys,intlevel)
    rawdata_encoded=pad_sequence(rawdata_encoded,batch_first=True)
    if shuffle:
        randord=torch.randperm(len(rawdata_encoded))
        return rawdata_encoded[randord],encoded_keys
    else:
        return rawdata_encoded,encoded_keys
    
def forcePadding(padded_data,padding,crop=False):
    if padding<=padded_data.shape[-1]: return padded_data if not crop else padded_data[:,:padding]
    extzeros=torch.zeros((len(padded_data),padding-padded_data.shape[-1])).to(padded_data.device)
    return torch.cat((padded_data,extzeros),dim=1)
    

def loadPDBsForTraining(filenames,N,S,shuffle=True,skipHs=False,keys=dict()): #Quick shortcut
    rawdata=readPDBFiles(filenames,N,S,skipHs=skipHs)
    return rawdataToDataset(rawdata,shuffle=shuffle,keys=keys)